In [ ]:
 pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 4.9 MB/s 


In [ ]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

import numpy as np 
from math import log 

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re

Baca Data

In [ ]:
#Data tweet
tweet = pd.read_csv('Data Tweet Eksperimen 1.csv', encoding='utf-8')

Cleaning

In [ ]:
#Function menghapus angka dan tanda baca
def removeNonWord(doc): 
    
    #hapus seluruh kata yang mengandung angka, contoh: perempuan2 
    result = doc.replace({'\w*\d\w*': ''}, regex=True)

    #menghilangkan URL
    result = result.replace({'https:\/\/.*': ' '}, regex=True) 

    #menghapus hashtag
    result = result.replace({'#([^\s]+)': ' '}, regex=True)
 
    #hapus seluruh karakter yang tidak termasuk alphabet 
    result = result.replace({'[\W_]+': ' '}, regex=True) 

    #menghapus mention
    result = result.replace({'@[\w\d]*': ' '}, regex=True)

    #hilangin retweet
    result = result.replace({'^rt[\s]+': ' '}, regex=True)
 
    #remove null dari hasil final 
    return result[result.notnull()] 

In [ ]:
#Lowercasing dan Menghapus angka&tanda baca
#tweet_lower = tweet['Tweet'].str.lower()
tweet_lower = removeNonWord(tweet['Tweet'].str.lower())

Tokenizing

In [ ]:
#Tokenizing
tweet_token = []

for doc in tweet_lower :
  tweet_token.append(word_tokenize(doc))

In [ ]:
tweet_token

Normalisasi

In [ ]:
def normalization(tokens) :
  kamus = {}
  with open('Kamus.txt') as kamus_normalisasi :
    for line in kamus_normalisasi :
      (key, val) = line.split("\t")
      kamus[key] = val
  tweet_normalization = []
  for word in tokens:
    if word in kamus :
      replace = kamus.get(word)
      tweet_normalization.append(replace.strip())
    else:
      tweet_normalization.append(word)
  return tweet_normalization

In [ ]:
#Normalization
tweet_norm = []

for doc in tweet_token:
  tweet_norm.append(normalization(doc))

In [ ]:
tweet_norm

Catatan : Salah satu tantangan dalam analisa text preprocessing menggunakan bhs indo adalah proses normalisasi kata harus detail, karena belum lengkap khususnya kata-kata spesifik kedaerahan, singkatan

Stopword Removal

In [ ]:
#Stopword Removal
listStopword =  nltk.corpus.stopwords.words('indonesian')
new_words=('besok','senin' , 'sekolah', 'kerja', 'tugas', 'kuliah')
for i in new_words:
    listStopword.append(i)

tweet_remove = []
temp_stop = ''
for i in range(len(tweet_norm)):
  for token in tweet_norm[i]:
    if token not in listStopword:
      temp_stop = temp_stop + token + ' '
  tweet_remove.append(temp_stop)
  temp_stop = ''

kata : haha, wkwkwk = dihapus

In [ ]:
tweet_remove

Stemming

In [ ]:
#Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

tweet_stemmed = [stemmer.stem(sentence) for sentence in tweet_remove]

In [ ]:
tweet_stemmed

In [ ]:
hasil = pd.DataFrame(tweet_stemmed)

In [ ]:
hasil

,0
0,malas banget
1,takut semenjak seram tidak tahu habis minggu t...
2,teman semangat nya semangat nya semangat jalan...
3,alhamdulillah syukur selagi keluh syukur kerja...
4,libur hadeuh
...,...
2146,judul kelompok dikerjainnya orang si posisi si...
2147,kasi marahin guru mampus
2148,malas ngadepin hidup teman bangsat blacklist t...
2149,sayang goodbye to horizontal body semester sia...


In [ ]:
hasil.set_axis(['Tweet'], axis='columns', inplace=True)

In [ ]:
hasil

,Tweet
0,malas banget
1,takut semenjak seram tidak tahu habis minggu t...
2,teman semangat nya semangat nya semangat jalan...
3,alhamdulillah syukur selagi keluh syukur kerja...
4,libur hadeuh
...,...
2146,judul kelompok dikerjainnya orang si posisi si...
2147,kasi marahin guru mampus
2148,malas ngadepin hidup teman bangsat blacklist t...
2149,sayang goodbye to horizontal body semester sia...


In [ ]:
pip install contractions

In [ ]:
import contractions

In [ ]:
hasil['Data'] = hasil['Tweet'].apply(lambda x: [contractions.fix(word) for word in x.split()])
hasil

,Tweet,Data
0,malas banget,"[malas, banget]"
1,takut semenjak seram tidak tahu habis minggu t...,"[takut, semenjak, seram, tidak, tahu, habis, m..."
2,teman semangat nya semangat nya semangat jalan...,"[teman, semangat, nya, semangat, nya, semangat..."
3,alhamdulillah syukur selagi keluh syukur kerja...,"[alhamdulillah, syukur, selagi, keluh, syukur,..."
4,libur hadeuh,"[libur, hadeuh]"
...,...,...
2146,judul kelompok dikerjainnya orang si posisi si...,"[judul, kelompok, dikerjainnya, orang, si, pos..."
2147,kasi marahin guru mampus,"[kasi, marahin, guru, mampus]"
2148,malas ngadepin hidup teman bangsat blacklist t...,"[malas, ngadepin, hidup, teman, bangsat, black..."
2149,sayang goodbye to horizontal body semester sia...,"[sayang, goodbye, to, horizontal, body, semest..."


In [ ]:
hasil['Data'].to_excel('Hasil Preprocessing.xlsx', index=False)